

# Hate Detection: BERT

## Author: Rami Abulfadl

The BERT model is fine tuned for binray classifier for hate detection using the dataset [Twitter hate speech] which can be downloaded from Kaggle´s competition with this link(https://www.kaggle.com/vkrahul/twitter-hate-speech?select=train_E6oV3lV.csv) in which tweets are identified as hateful by internet users and compiled by Hatebase.org based on Davidson et al. (https://arxiv.org/pdf/1703.04009.pdf)

### Load the required dependencies

In [ ]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
!pip install sentencepiece

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from sklearn import model_selection
from sklearn import metrics
import keras

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

import tokenization

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/My Drive/TFMColab/hate/train.csv")
val = pd.read_csv("/content/drive/My Drive/TFMColab/hate/dev.csv")
test = pd.read_csv("/content/drive/My Drive/TFMColab/hate/test.csv")

### Define Helper Functions
[Source](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub) of helper functions.

In [ ]:
def bert_encode(texts, tokenizer, max_len=160):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=160):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])
    
    return model

### Load BERT from the Tensorflow Hub

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 10.5 s, sys: 1.85 s, total: 12.4 s
Wall time: 12.2 s


### Load CSV files containing data

### Load tokenizer

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

### Text encoding

In [ ]:
train_input = bert_encode(train.Phrase.values, tokenizer, max_len = 160)
test_input = bert_encode(test.Phrase.values, tokenizer, max_len = 160)
val_input = bert_encode(val.Phrase.values, tokenizer, max_len = 160)



train_labels = train.sentiment_values.values
test_labels = test.sentiment_values.values
val_labels = val.sentiment_values.values

### Build model

In [ ]:
model = build_model(bert_layer, max_len = 160)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

### Save the best model and early stopping

To prevent the model from overfitting early stopping has been enabled.

Early stopping is a method that allows us to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out/validation dataset.


In [ ]:
# Save the model after every epoch.
saveBestModel = ModelCheckpoint('/content/drive/My Drive/TFMColab/hate/best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

### Fit the model


In [ ]:
train_history = model.fit(
    train_input, train_labels,
    validation_data=(val_input, val_labels),
    epochs=7,
    batch_size=14,
    callbacks=[saveBestModel, earlyStopping]
)

#model.save('model.h5')

Epoch 1/7
1824/1824 [==============================] - ETA: 0s - loss: 0.1347 - accuracy: 0.9516 - precision: 0.7543 - recall: 0.4618 - true_positives: 829.0000WARNING:tensorflow:Can save best model only with val_acc available, skipping.


1824/1824 [==============================] - 2769s 2s/step - loss: 0.1347 - accuracy: 0.9516 - precision: 0.7543 - recall: 0.4618 - true_positives: 829.0000 - val_loss: 0.1044 - val_accuracy: 0.9613 - val_precision: 0.7751 - val_recall: 0.6009 - val_true_positives: 131.0000
Epoch 2/7
1824/1824 [==============================] - ETA: 0s - loss: 0.0668 - accuracy: 0.9764 - precision: 0.8916 - recall: 0.7560 - true_positives: 1357.0000WARNING:tensorflow:Can save best model only with val_acc available, skipping.


1824/1824 [==============================] - 2768s 2s/step - loss: 0.0668 - accuracy: 0.9764 - precision: 0.8916 - recall: 0.7560 - true_positives: 1357.0000 - val_loss: 0.1053 - val_accuracy: 0.9656 - val_precision: 0.8497 - val_recall: 0.5963 - val_true_positives: 130.0000
Epoch 3/7
1824/1824 [==============================] - ETA: 0s - loss: 0.0225 - accuracy: 0.9930 - precision: 0.9681 - recall: 0.9309 - true_positives: 1671.0000WARNING:tensorflow:Can save best model only with val_acc available, skipping.


1824/1824 [==============================] - 2769s 2s/step - loss: 0.0225 - accuracy: 0.9930 - precision: 0.9681 - recall: 0.9309 - true_positives: 1671.0000 - val_loss: 0.1462 - val_accuracy: 0.9622 - val_precision: 0.8750 - val_recall: 0.5138 - val_true_positives: 112.0000
Epoch 4/7
1824/1824 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.9989 - precision: 0.9950 - recall: 0.9900 - true_positives: 1777.0000WARNING:tensorflow:Can save best model only with val_acc available, skipping.


1824/1824 [==============================] - 2768s 2s/step - loss: 0.0040 - accuracy: 0.9989 - precision: 0.9950 - recall: 0.9900 - true_positives: 1777.0000 - val_loss: 0.1491 - val_accuracy: 0.9631 - val_precision: 0.7676 - val_recall: 0.6514 - val_true_positives: 142.0000


### Evaluate model results with test data

Results were obtained by using the 'predict' function.

In [ ]:
test_pred = model.predict(test_input)
test_pred = test_pred.round().astype(int)

In [ ]:
recall = metrics.recall_score(test_labels,test_pred)
precision = metrics.precision_score(test_labels,test_pred)
f1_score = metrics.f1_score(test_labels,test_pred)
accuracy = metrics.accuracy_score(test_labels,test_pred)
loss = metrics.log_loss(test_labels,test_pred)


In [ ]:
print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 1.2077359158654162
Accuracy: 0.9650327817670934
Precision: 0.7853658536585366
Recall: 0.7030567685589519
f1 score: 0.7419354838709677


In [ ]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# kappa
kappa = cohen_kappa_score(test_labels,test_pred)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(test_labels,test_pred)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(test_labels,test_pred)
print(matrix)

Cohens kappa: 0.723243
ROC AUC: 0.844131
[[2930   44]
 [  68  161]]


### Extract False Positives and False Negatives

False Positives and False Negatives are stored in a CSV file for posterior analysis. 

In [ ]:
def getFP_FN_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    for i in range(len(test_y)):
        if(pred_y[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['Phrase'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['Phrase'][test_y.index[i]])
            FN_index.append(test_y.index[i])
            
    return FP_text,FP_index,FN_text,FN_index

In [ ]:
'''Returns 2 dataframes, one with all the False Positives and one with all the False Negatives'''
def getFP_FN(test_X, test_y, pred_y):
    FP_text,FP_index,FN_text,FN_index = getFP_FN_lists(test_X, test_y, pred_y)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
    
    return df_FP,df_FN

In [ ]:
# We get the FPs and FNs as DataFrames and store them to CSVs
df_FP,df_FN = getFP_FN(test['Phrase'], test['sentiment_values'],test_pred)
df_FP.to_csv('/content/drive/My Drive/TFMColab/hate/bert_FP_bert.csv', index=True)
df_FN.to_csv('/content/drive/My Drive/TFMColab/hate/bert_FN_bert.csv', index=True)